In [0]:
!pip install PyDrive # 安装一个包


    100% |████████████████████████████████| 993kB 10.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
# 以下两句代码在 Google Colab 才需要添加，其他平台忽略
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# # 2. Auto-iterate using the query syntax
# #    https://developers.google.com/drive/v2/web/search-parameters
# file_list = drive.ListFile(
#     {'q': "'https://drive.google.com/open?id=1AFxy5ICODGYsdtDYfBzJK6AiVlxyjPHx' in parents"}).GetList()

# for f in file_list:
#   # 3. Create & download by id.
#   print('title: %s, id: %s' % (f['title'], f['id']))
#   fname = os.path.join(local_download_path, f['title'])
#   print('downloading to {}'.format(fname))
#   f_ = drive.CreateFile({'id': f['id']})
#   f_.GetContentFile(fname)

In [0]:
download = drive.CreateFile({'id':'1AFxy5ICODGYsdtDYfBzJK6AiVlxyjPHx'})
download.GetContentFile('train.txt')

In [0]:
!ls

adc.json  sample_data  train.txt


In [0]:
import pandas as pd

In [0]:
train_df = pd.read_csv('train.txt', sep='\t', header=None)

In [0]:
train_df.head()

,0,1
0,娱乐,《青蛇》造型师默认新《红楼梦》额妆抄袭（图） 凡是看过电影《青蛇》的人，都不会忘记青白二蛇的...
1,娱乐,６．１６日剧榜 ＜最后的朋友＞ 亮最后杀招成功登顶 《最后的朋友》本周的电视剧排行榜单依然只...
2,娱乐,超乎想象的好看《纳尼亚传奇２：凯斯宾王子》 现时资讯如此发达，搜狐电影评审团几乎人人在没有看...
3,娱乐,吴宇森：赤壁大战不会出现在上集 “希望《赤壁》能给你们不一样的感觉。”对于自己刚刚拍完的影片...
4,娱乐,组图：《多情女人痴情男》陈浩民现场耍宝 陈浩民：外面的朋友大家好，现在是搜狐现场直播，欢迎《...


In [0]:
download = drive.CreateFile({'id':'1BmP9IbcJK-XdnKnsP7Ta6cNwZsYpewWt'})
download.GetContentFile('test.txt')

In [0]:
test_df = pd.read_csv('test.txt', sep='\t', header=None)

In [0]:
train_df.columns = ['分类', '文章']

In [0]:
download = drive.CreateFile({'id':'1rOI0ufmOz9SWBqt6quFY7xikffqyxh-N'})
download.GetContentFile('stopwords.txt')

In [0]:
with open('stopwords.txt', encoding='utf8') as file:
    stopWord_list = [k.strip() for k in file.readlines()]

In [0]:
import jieba
import time

In [0]:
stopword_list = [k.strip() for k in open('stopwords.txt', encoding='utf8').readlines() if k.strip() != '']
cutWords_list = []
i = 0
startTime = time.time()
for article in train_df['文章']:
    cutWords = [k for k in jieba.cut(article) if k not in stopword_list]
    i += 1
    if i % 1000 == 0:
        print('前%d篇文章分词共花费%.2f秒' %(i, time.time()-startTime))
    cutWords_list.append(cutWords)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.975 seconds.
Prefix dict has been built succesfully.


前1000篇文章分词共花费47.18秒
前2000篇文章分词共花费91.91秒
前3000篇文章分词共花费185.69秒
前4000篇文章分词共花费275.88秒
前5000篇文章分词共花费361.28秒
前6000篇文章分词共花费451.05秒
前7000篇文章分词共花费475.68秒
前8000篇文章分词共花费500.67秒
前9000篇文章分词共花费538.80秒
前10000篇文章分词共花费575.11秒
前11000篇文章分词共花费617.60秒
前12000篇文章分词共花费663.64秒
前13000篇文章分词共花费691.41秒
前14000篇文章分词共花费717.52秒
前15000篇文章分词共花费753.14秒
前16000篇文章分词共花费789.96秒
前17000篇文章分词共花费825.81秒
前18000篇文章分词共花费860.11秒
前19000篇文章分词共花费941.06秒
前20000篇文章分词共花费1022.69秒
前21000篇文章分词共花费1042.51秒
前22000篇文章分词共花费1062.79秒
前23000篇文章分词共花费1094.27秒
前24000篇文章分词共花费1125.36秒


In [0]:
with open('cutWords_list.txt', 'w') as file: 
    for cutWords in cutWords_list:
        file.write(' '.join(cutWords) + '\n')

In [0]:
!ls

adc.json  cutWords_list.txt  sample_data  stopwords.txt  test.txt  train.txt


In [0]:
file.close()

In [0]:
with open('cutWords_list.txt') as file:
    cutWords_list = [k.split() for k in file.readlines()]

In [0]:
import numpy as np

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
stopword_list = [k.strip() for k in open('stopwords.txt', encoding='utf8').readlines() if k.strip() != '']

In [0]:
#tfv = TfidfVectorizer(min_df=2,  max_features=None,  ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1, stop_words = stopword_list)
tfv = TfidfVectorizer(cutWords_list, stop_words=stopword_list, min_df=40, max_df=0.3)

In [0]:
temp_inputlist = [" ".join(x) for x in cutWords_list]

In [0]:
tfv.fit(temp_inputlist)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['nbsp'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8',
        input=[['青蛇', '造型师', '默认', '新', '红楼梦', '额妆', '抄袭', '图', '看过', '电影', '青蛇', '不会', '忘记', '青白', '二蛇', '经典', '造型', '飘逸', '身材', '妩媚', '妆容', '独特', '取材于', '京剧', '女角', '头型', '精华', '处', '片子', '称之为', '额妆', '１', '３', '年', '今天', '新版', '红楼梦', '定妆', '照里', '看到', '这种', '熟悉', '妆容', '片子', '昨日', '知情人', '记者', '报料', '叶锦添...销', '车旅费', '上调', '手机', '报销', '费用', '灾区', '员工', '安抚', '慰问', '措施', '紧锣密鼓', '筹划', '之中', '责任编辑', '赵文琳']],
        lowercase=True, max_df=0.3, max_features=None, min_df=40,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['aboard', 'about', 'about', 'about', 'above', 'above', 'according', 'according to', 'across', 'across', 'afore', 'after', 'after', 'after', 'afterwards', 'again', 'against', 'against', 'against', 'agin', 'all', 'all', 'almost', 'almost', 'alone', 'along', 'along', 'alongside', 'already',... '）', '，

In [0]:
X_all = tfv.transform(temp_inputlist)

In [0]:
print('词表大小:', len(tfv.vocabulary_))
print(X_all.shape)

词表大小: 27752
(24000, 27752)


In [0]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
train_df = pd.read_csv('train.txt', sep='\t', header=None)
labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(train_df[0])

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X_all, y, test_size=0.2)
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
logistic_model.fit(train_X, train_y)
logistic_model.score(test_X, test_y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.8589583333333334

In [0]:
import pickle

with open('tfv.model', 'wb') as file:
    save = {
        'labelEncoder' : labelEncoder,
        'tfidfVectorizer' : tfv,
        'logistic_model' : logistic_model
    }
    pickle.dump(save, file)

In [0]:
!ls -a

.   adc.json  cutWords_list.txt  stopwords.txt	tfidf.model  train.txt
..  .config   sample_data	 test.txt	tfv.model


In [0]:
import pickle

with open('tfv.model', 'rb') as file:
    tfidf_model = pickle.load(file)
    tfidfVectorizer = tfidf_model['tfidfVectorizer']
    labelEncoder = tfidf_model['labelEncoder']
    logistic_model = tfidf_model['logistic_model']

In [0]:
train_df = pd.read_csv('train.txt', sep='\t', header=None)
X = tfidfVectorizer.transform(train_df[1])
y = labelEncoder.transform(train_df[0])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['nbsp'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
cv_split = ShuffleSplit(n_splits=5, test_size=0.3)
score_ndarray = cross_val_score(logistic_model, X, y, cv=cv_split)
print(score_ndarray)
print(score_ndarray.mean())

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[0.81972222 0.81666667 0.81527778 0.81972222 0.82194444]
0.8186666666666665


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix
import pandas as pd

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
logistic_model = LogisticRegressionCV(multi_class='multinomial', solver='lbfgs')
logistic_model.fit(train_X, train_y)
predict_y = logistic_model.predict(test_X)
pd.DataFrame(confusion_matrix(test_y, predict_y), 
             columns=labelEncoder.classes_, 
             index=labelEncoder.classes_)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase t

,体育,健康,女人,娱乐,房地产,教育,文化,新闻,旅游,汽车,科技,财经
体育,385,1,10,5,1,6,12,1,3,2,4,5
健康,4,308,9,3,4,9,13,5,14,1,6,6
女人,3,7,320,16,3,3,12,2,3,3,1,0
娱乐,4,2,6,371,0,2,14,2,4,1,3,2
房地产,3,7,7,5,321,5,4,9,7,2,8,19
教育,1,7,4,0,2,332,16,3,9,4,3,4
文化,3,7,10,16,2,4,326,6,7,3,6,1
新闻,3,10,8,2,5,3,8,362,10,1,4,11
旅游,2,14,7,5,15,3,21,8,329,1,10,0
汽车,1,3,7,0,5,4,3,2,4,374,5,2


In [0]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

def eval_model(y_true, y_pred, labels):
    # 计算每个分类的Precision, Recall, f1, support
    p, r, f1, s = precision_recall_fscore_support(y_true, y_pred)
    # 计算总体的平均Precision, Recall, f1, support
    tot_p = np.average(p, weights=s)
    tot_r = np.average(r, weights=s)
    tot_f1 = np.average(f1, weights=s)
    tot_s = np.sum(s)
    res1 = pd.DataFrame({
        u'Label': labels,
        u'Precision': p,
        u'Recall': r,
        u'F1': f1,
        u'Support': s
    })
    res2 = pd.DataFrame({
        u'Label': ['总体'],
        u'Precision': [tot_p],
        u'Recall': [tot_r],
        u'F1': [tot_f1],
        u'Support': [tot_s]
    })
    res2.index = [999]
    res = pd.concat([res1, res2])
    return res[['Label', 'Precision', 'Recall', 'F1', 'Support']]

predict_y = logistic_model.predict(test_X)
eval_model(test_y, predict_y, labelEncoder.classes_)

,Label,Precision,Recall,F1,Support
0,体育,0.934917,0.905000,0.919715,1000
1,健康,0.789687,0.781000,0.785319,1000
2,女人,0.824427,0.864000,0.843750,1000
3,娱乐,0.876228,0.892000,0.884044,1000
4,房地产,0.839468,0.821000,0.830131,1000
5,教育,0.862845,0.843000,0.852807,1000
6,文化,0.760967,0.850000,0.803023,1000
7,新闻,0.827795,0.822000,0.824887,1000
8,旅游,0.838437,0.794000,0.815614,1000
9,汽车,0.942736,0.889000,0.915080,1000


In [0]:
import pandas as pd

test_df = pd.read_csv('test.txt', sep='\t', header=None)
test_X = tfidfVectorizer.transform(test_df[1])
test_y = labelEncoder.transform(test_df[0])
predict_y = logistic_model.predict(test_X)
eval_model(test_y, predict_y, labelEncoder.classes_)

,Label,Precision,Recall,F1,Support
0,体育,0.934917,0.905000,0.919715,1000
1,健康,0.789687,0.781000,0.785319,1000
2,女人,0.824427,0.864000,0.843750,1000
3,娱乐,0.876228,0.892000,0.884044,1000
4,房地产,0.839468,0.821000,0.830131,1000
5,教育,0.862845,0.843000,0.852807,1000
6,文化,0.760967,0.850000,0.803023,1000
7,新闻,0.827795,0.822000,0.824887,1000
8,旅游,0.838437,0.794000,0.815614,1000
9,汽车,0.942736,0.889000,0.915080,1000
